In [1]:
import torch

c:\Users\andre\anaconda3\envs\elsr\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss /= len(train_loader.dataset)
    accuracy = 100. * correct / total
    return train_loss, accuracy

In [3]:
import torch

def evaluate(device, model, dataloader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from models.mlp_wd import MLPWD



EPOCHS = 10
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLPWD().to(device)
print("LinW layers:", "\n".join([f"Depth {model[i].depth}: {model[i]}" for i in range(len(model))]), sep="\n\n")
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()


c:\Users\andre\anaconda3\envs\elsr\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] Impossibile trovare la procedura specificata
  warn(f"Failed to load image Python extension: {e}")


LinW layers:

Depth 0: LinW(in_features=256, out_features=256, bias=True)
Depth 1: LinW(in_features=256, out_features=256, bias=True)
Epoch 1/10, Training Loss: 0.5630, Training Accuracy: 81.56%, Test accuracy: 93.19%
Epoch 2/10, Training Loss: 0.1719, Training Accuracy: 95.08%, Test accuracy: 96.37%
Epoch 3/10, Training Loss: 0.1127, Training Accuracy: 96.79%, Test accuracy: 97.05%
Epoch 4/10, Training Loss: 0.0813, Training Accuracy: 97.64%, Test accuracy: 96.82%
Epoch 5/10, Training Loss: 0.0611, Training Accuracy: 98.25%, Test accuracy: 97.54%
Epoch 6/10, Training Loss: 0.0476, Training Accuracy: 98.61%, Test accuracy: 97.15%
Epoch 7/10, Training Loss: 0.0383, Training Accuracy: 98.86%, Test accuracy: 97.80%
Epoch 8/10, Training Loss: 0.0338, Training Accuracy: 99.02%, Test accuracy: 97.64%
Epoch 9/10, Training Loss: 0.0268, Training Accuracy: 99.17%, Test accuracy: 97.81%
Epoch 10/10, Training Loss: 0.0228, Training Accuracy: 99.35%, Test accuracy: 97.76%
